In [1]:
import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import r2_score,accuracy_score,confusion_matrix,precision_score
from sklearn.metrics import recall_score,f1_score

In [2]:
df = pd.read_csv('../data/LUCAS_Topsoil_2015.csv')

In [3]:
df.head()

,Point_ID,Revisited_point,Coarse,Clay,Sand,Silt,pH(CaCl2),pH(H2O),EC,OC,...,K,LC1,LU1,Soil_Stones,NUTS_0,NUTS_1,NUTS_2,NUTS_3,LC1_Desc,LU1_Desc
0,34103754,No,NaN,NaN,NaN,NaN,3.9,3.91,44.20,25.5,...,24.6,H11,U420,1,UK,UKM,UKM8,UKM81,Inland marshes,Semi-natural and natural areas not in use
1,34443774,No,NaN,NaN,NaN,NaN,3.1,3.91,46.40,503.5,...,460.3,H12,U420,1,UK,UKM,UKM7,UKM77,Peatbogs,Semi-natural and natural areas not in use
2,35163814,No,NaN,NaN,NaN,NaN,4.9,5.48,15.85,51.4,...,173.2,H11,U420,1,UK,UKM,UKM7,UKM71,Inland marshes,Semi-natural and natural areas not in use
3,32323656,No,NaN,NaN,NaN,NaN,3.0,3.76,26.90,470.3,...,313.0,H12,U150,1,UK,UKN,UKN1,UKN10,Peatbogs,OTHER PRIMARY PRODUCTION
4,34463934,No,28.0,10.0,46.0,44.0,3.9,4.04,28.40,43.1,...,38.6,D20,U111,2,UK,UKM,UKM6,UKM61,Shrubland without tree cover,Agriculture (excluding fallow land and kitchen...


In [4]:
df.tail()

,Point_ID,Revisited_point,Coarse,Clay,Sand,Silt,pH(CaCl2),pH(H2O),EC,OC,...,K,LC1,LU1,Soil_Stones,NUTS_0,NUTS_1,NUTS_2,NUTS_3,LC1_Desc,LU1_Desc
21854,45402772,Yes,NaN,NaN,NaN,NaN,6.9,7.03,81.0,49.9,...,200.1,E20,U111,3,AT,AT3,AT32,AT323,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
21855,47722672,Yes,NaN,NaN,NaN,NaN,5.7,5.98,22.3,19.4,...,244.9,E20,U111,1,AT,AT2,AT22,AT224,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
21856,45882618,Yes,NaN,NaN,NaN,NaN,6.8,6.99,65.1,53.9,...,90.0,E20,U111,1,AT,AT2,AT21,AT212,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
21857,45342706,Yes,NaN,NaN,NaN,NaN,6.0,6.12,30.7,24.2,...,32.7,E20,U111,1,AT,AT3,AT32,AT322,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
21858,47742660,Yes,NaN,NaN,NaN,NaN,4.2,4.68,7.3,26.0,...,162.0,C10,U120,1,AT,AT2,AT22,AT224,Broadleaved woodland,FORESTRY


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21859 entries, 0 to 21858
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Point_ID         21859 non-null  int64  
 1   Revisited_point  21859 non-null  object 
 2   Coarse           4258 non-null   float64
 3   Clay             4260 non-null   float64
 4   Sand             4260 non-null   float64
 5   Silt             4260 non-null   float64
 6   pH(CaCl2)        21859 non-null  float64
 7   pH(H2O)          21859 non-null  float64
 8   EC               21859 non-null  float64
 9   OC               21859 non-null  float64
 10  CaCO3            21859 non-null  int64  
 11  P                21859 non-null  float64
 12  N                21859 non-null  float64
 13  K                21859 non-null  float64
 14  LC1              21859 non-null  object 
 15  LU1              21859 non-null  object 
 16  Soil_Stones      21859 non-null  int64  
 17  NUTS_0      

In [6]:
df.describe()

,Point_ID,Coarse,Clay,Sand,Silt,pH(CaCl2),pH(H2O),EC,OC,CaCO3,P,N,K,Soil_Stones
count,2.185900e+04,4258.000000,4260.000000,4260.000000,4260.000000,21859.000000,21859.000000,21859.000000,21859.000000,21859.000000,21859.000000,21859.000000,21859.000000,21859.000000
mean,4.287592e+07,21.633866,19.207981,37.418779,43.367136,5.745199,6.130833,26.091462,43.275982,57.268173,32.613839,3.104685,195.529677,1.406835
std,8.177514e+06,14.804102,10.723485,19.170218,12.544561,1.398392,1.345302,33.339621,76.696684,135.300507,32.372183,3.672019,232.383766,0.791719
min,2.658177e+07,0.000000,0.000000,2.000000,0.000000,2.600000,3.170000,0.310000,0.100000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.538292e+07,11.000000,11.000000,22.000000,35.000000,4.500000,4.920000,10.295000,12.500000,0.000000,11.200000,1.300000,77.500000,1.000000
50%,4.438403e+07,19.000000,18.000000,34.000000,45.000000,5.800000,6.070000,17.240000,20.400000,1.000000,23.800000,2.000000,143.100000,1.000000
75%,4.960328e+07,30.000000,26.000000,50.000000,53.000000,7.100000,7.450000,28.400000,38.600000,19.000000,43.700000,3.300000,245.600000,2.000000
max,6.498167e+07,90.000000,62.000000,100.000000,72.000000,10.000000,10.370000,969.000000,560.200000,976.000000,1017.600000,38.500000,10030.900000,4.000000


In [7]:
df.corr()

,Point_ID,Coarse,Clay,Sand,Silt,pH(CaCl2),pH(H2O),EC,OC,CaCO3,P,N,K,Soil_Stones
Point_ID,1.000000,-0.247229,-0.010075,-0.039549,0.069144,-0.163294,-0.170245,-0.092160,0.081153,-0.237338,0.001639,0.050536,-0.043450,-0.175487
Coarse,-0.247229,1.000000,0.020987,-0.055709,0.067211,0.103743,0.102403,0.027431,0.116495,0.138949,-0.072721,0.133731,0.020227,0.336909
Clay,-0.010075,0.020987,1.000000,-0.789135,0.351458,0.564601,0.562431,-0.086244,-0.294735,0.385466,-0.161841,-0.233193,0.281231,0.050961
Sand,-0.039549,-0.055709,-0.789135,1.000000,-0.851861,-0.359668,-0.331972,-0.091682,0.116775,-0.197842,0.109576,-0.023481,-0.212525,0.001842
Silt,0.069144,0.067211,0.351458,-0.851861,1.000000,0.067389,0.027000,0.213315,0.072935,-0.027062,-0.029481,0.234535,0.083988,-0.045467
pH(CaCl2),-0.163294,0.103743,0.564601,-0.359668,0.067389,1.000000,0.988969,0.037117,-0.355304,0.514421,-0.112162,-0.278667,0.291199,0.062023
pH(H2O),-0.170245,0.102403,0.562431,-0.331972,0.027000,0.988969,1.000000,0.001128,-0.328945,0.543443,-0.122150,-0.276620,0.292024,0.072444
EC,-0.092160,0.027431,-0.086244,-0.091682,0.213315,0.037117,0.001128,1.000000,0.366723,0.077438,0.249799,0.502906,0.177682,-0.036653
OC,0.081153,0.116495,-0.294735,0.116775,0.072935,-0.355304,-0.328945,0.366723,1.000000,-0.114602,0.242309,0.908556,0.079637,-0.016282
CaCO3,-0.237338,0.138949,0.385466,-0.197842,-0.027062,0.514421,0.543443,0.077438,-0.114602,1.000000,-0.111456,-0.114324,0.154423,0.153395


In [8]:
df.cov()

,Point_ID,Coarse,Clay,Sand,Silt,pH(CaCl2),pH(H2O),EC,OC,CaCO3,P,N,K,Soil_Stones
Point_ID,6.687174e+13,-3.094874e+07,-913536.888917,-6.411003e+06,7.334549e+06,-1.867330e+06,-1.872909e+06,-2.512593e+07,5.089815e+07,-2.625958e+08,433809.057903,1.517481e+06,-8.256873e+07,-1.136154e+06
Coarse,-3.094874e+07,2.191614e+02,3.332189,-1.581094e+01,1.248295e+01,2.101301e+00,2.052625e+00,1.332168e+01,1.018408e+02,3.478689e+02,-34.423252,6.679769e+00,7.203614e+01,5.027266e+00
Clay,-9.135369e+05,3.332189e+00,114.993127,-1.622238e+02,4.727864e+01,8.283394e+00,8.165594e+00,-3.033310e+01,-1.866062e+02,6.989020e+02,-55.496930,-8.435707e+00,7.253761e+02,5.507590e-01
Sand,-6.411003e+06,-1.581094e+01,-162.223770,3.674973e+02,-2.048572e+02,-9.433206e+00,-8.616108e+00,-5.764559e+01,1.321704e+02,-6.412667e+02,67.171929,-1.518475e+00,-9.799435e+02,3.558838e-02
Silt,7.334549e+06,1.248295e+01,47.278638,-2.048572e+02,1.573660e+02,1.156574e+00,4.585598e-01,8.776693e+01,5.401911e+01,-5.739992e+01,-11.826218,9.925057e+00,2.534176e+02,-5.748296e-01
pH(CaCl2),-1.867330e+06,2.101301e+00,8.283394,-9.433206e+00,1.156574e+00,1.955500e+00,1.860507e+00,1.730458e+00,-3.810703e+01,9.733004e+01,-5.077440,-1.430936e+00,9.462894e+01,6.866810e-02
pH(H2O),-1.872909e+06,2.052625e+00,8.165594,-8.616108e+00,4.585598e-01,1.860507e+00,1.809837e+00,5.060251e-02,-3.394059e+01,9.891751e+01,-5.319679,-1.366498e+00,9.129449e+01,7.716011e-02
EC,-2.512593e+07,1.332168e+01,-30.333102,-5.764559e+01,8.776693e+01,1.730458e+00,5.060251e-02,1.111530e+03,9.377236e+02,3.493117e+02,269.602442,6.156758e+01,1.376609e+03,-9.674816e-01
OC,5.089815e+07,1.018408e+02,-186.606239,1.321704e+02,5.401911e+01,-3.810703e+01,-3.394059e+01,9.377236e+02,5.882381e+03,-1.189238e+03,601.613857,2.558782e+02,1.419374e+03,-9.886729e-01
CaCO3,-2.625958e+08,3.478689e+02,698.901984,-6.412667e+02,-5.739992e+01,9.733004e+01,9.891751e+01,3.493117e+02,-1.189238e+03,1.830623e+04,-488.175863,-5.679936e+01,4.855321e+03,1.643166e+01


In [9]:
df.isnull().sum()

Point_ID               0
Revisited_point        0
Coarse             17601
Clay               17599
Sand               17599
Silt               17599
pH(CaCl2)              0
pH(H2O)                0
EC                     0
OC                     0
CaCO3                  0
P                      0
N                      0
K                      0
LC1                    0
LU1                    0
Soil_Stones            0
NUTS_0                 0
NUTS_1                 0
NUTS_2                 0
NUTS_3                 0
LC1_Desc               0
LU1_Desc               0
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4258 entries, 4 to 21025
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Point_ID         4258 non-null   int64  
 1   Revisited_point  4258 non-null   object 
 2   Coarse           4258 non-null   float64
 3   Clay             4258 non-null   float64
 4   Sand             4258 non-null   float64
 5   Silt             4258 non-null   float64
 6   pH(CaCl2)        4258 non-null   float64
 7   pH(H2O)          4258 non-null   float64
 8   EC               4258 non-null   float64
 9   OC               4258 non-null   float64
 10  CaCO3            4258 non-null   int64  
 11  P                4258 non-null   float64
 12  N                4258 non-null   float64
 13  K                4258 non-null   float64
 14  LC1              4258 non-null   object 
 15  LU1              4258 non-null   object 
 16  Soil_Stones      4258 non-null   int64  
 17  NUTS_0       

In [12]:
df['Revisited_point'] = df['Revisited_point'].replace(['Yes'],'1')
df['Revisited_point'] = df['Revisited_point'].replace(['No'],'0')

In [13]:
df = df[['Revisited_point', 'Coarse','Sand', 'Silt',
       'pH(CaCl2)', 'pH(H2O)', 'EC', 'OC', 'CaCO3', 'P', 'N', 'K', 'Elevation','Clay']]

KeyError: "['Elevation'] not in index"

In [14]:
df

,Point_ID,Revisited_point,Coarse,Clay,Sand,Silt,pH(CaCl2),pH(H2O),EC,OC,...,K,LC1,LU1,Soil_Stones,NUTS_0,NUTS_1,NUTS_2,NUTS_3,LC1_Desc,LU1_Desc
4,34463934,0,28.0,10.0,46.0,44.0,3.9,4.04,28.40,43.1,...,38.6,D20,U111,2,UK,UKM,UKM6,UKM61,Shrubland without tree cover,Agriculture (excluding fallow land and kitchen...
5,33983238,0,18.0,14.0,36.0,50.0,4.2,4.41,41.80,32.4,...,48.0,E20,U362,1,UK,UKL,UKL2,UKL22,Grassland without tree/shrub cover,Sport
6,34043240,0,20.0,18.0,35.0,46.0,4.9,5.13,32.00,21.1,...,36.0,E20,U111,1,UK,UKL,UKL2,UKL22,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
7,33723266,0,13.0,14.0,36.0,50.0,4.0,4.16,72.40,53.2,...,158.5,E20,U111,1,UK,UKL,UKL1,UKL17,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
8,34203268,0,34.0,19.0,48.0,34.0,3.7,3.87,11.63,16.0,...,24.4,E20,U420,1,UK,UKL,UKL1,UKL16,Grassland without tree/shrub cover,Semi-natural and natural areas not in use
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21021,64841666,1,6.0,23.0,55.0,22.0,7.0,7.44,31.30,6.4,...,589.4,B21,U111,1,CY,CY0,CY00,CY000,Potatoes,Agriculture (excluding fallow land and kitchen...
21022,64841670,1,17.0,48.0,21.0,30.0,7.4,8.02,19.54,11.2,...,835.6,B54,U111,2,CY,CY0,CY00,CY000,Mix of cereals,Agriculture (excluding fallow land and kitchen...
21023,64161658,1,15.0,33.0,34.0,32.0,7.6,8.10,19.12,5.8,...,337.1,F40,U111,3,CY,CY0,CY00,CY000,OTHER BARE SOIL,Agriculture (excluding fallow land and kitchen...
21024,63921638,1,23.0,19.0,46.0,35.0,7.0,7.50,10.34,5.2,...,56.2,D20,U420,3,CY,CY0,CY00,CY000,Shrubland without tree cover,Semi-natural and natural areas not in use


In [16]:
X = df[['Revisited_point', 'Coarse','Sand', 'Silt',
       'pH(CaCl2)', 'pH(H2O)', 'EC', 'OC', 'CaCO3', 'P', 'N', 'K']]

In [17]:
y = df['Clay']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [19]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear')
rfc = RandomForestClassifier(n_estimators=50, random_state=2,oob_score=True)
abc = AdaBoostClassifier(n_estimators=55, random_state=2)
bc = BaggingClassifier(n_estimators=100,oob_score=True)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
# gbdt = GradientBoostingClassifier(n_estimators=50,random_s

In [20]:
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
#     'GBDT':gbdt,
}

In [21]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = r2_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred,average='weighted',zero_division=1)
    
    return accuracy,precision

In [22]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    
    current_accuracy,current_precision = train_classifier(clf, X_train,y_train.values.ravel(),X_test,y_test)
    
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

For  SVC
Accuracy -  -0.22174532912747824
Precision -  0.9761828561352465
For  KN
Accuracy -  0.3780796604779877
Precision -  0.07481509944616598
For  DT
Accuracy -  0.8156109293837214
Precision -  0.5395547532683115
For  LR
Accuracy -  0.7162526999023374
Precision -  0.18017322830813873
For  RF
Accuracy -  0.8827887944638283
Precision -  0.18059009209708654
For  AdaBoost
Accuracy -  0.30355056996355634
Precision -  0.8954646644787491
For  BgC
Accuracy -  0.968203032843597
Precision -  0.3008763196883495
For  ETC
Accuracy -  0.8859309849272526
Precision -  0.19236304610912486


In [23]:
reg1=LogisticRegression(solver='liblinear')

In [24]:
reg1.fit(X_train,y_train)
y_pred = reg1.predict(X_test)
accuracy = r2_score(y_test,y_pred)

print("For ",reg1)
print("Accuracy - ",accuracy)

For  LogisticRegression(solver='liblinear')
Accuracy -  0.7162526999023374


In [25]:
from sklearn.svm import SVR
reg2 = SVR()

In [26]:
reg2.fit(X_train,y_train)
y_pred = reg2.predict(X_test)
accuracy = r2_score(y_test,y_pred)

print("For ",reg2)
print("Accuracy - ",accuracy)

For  SVR()
Accuracy -  0.6957291614635276


In [27]:
from sklearn.ensemble import BaggingRegressor
reg3 = BaggingRegressor(n_estimators=100)

In [28]:
reg3.fit(X_train,y_train)
y_pred = reg3.predict(X_test)
accuracy = r2_score(y_test,y_pred)

print("For ",reg3)
print("Accuracy - ",accuracy)

For  BaggingRegressor(n_estimators=100)
Accuracy -  0.9906652064088993


In [29]:
import matplotlib.pyplot as plt
%matplotlib inline